In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent, FakeUserAgentError
import pandas as pd
import time
from random import randint
import logging
from urllib.parse import urljoin

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class TripAdvisorScraper:
    def __init__(self, base_url):
        """
        Initialise le scraper.
        :param base_url: URL de base pour le scraping
        """
        self.base_url = base_url
        self.setup_user_agent()
        self.session = requests.Session()
        self.min_delay = 2
        self.max_delay = 5

    def setup_user_agent(self):
        """Initialise le User-Agent avec gestion des erreurs."""
        try:
            self.ua = UserAgent()
        except FakeUserAgentError:
            logger.warning("Impossible d'utiliser FakeUserAgent. Utilisation d'un User-Agent par défaut.")
            self.ua = None

    def get_headers(self):
        """Crée des en-têtes HTTP pour simuler un navigateur."""
        default_ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        return {
            "User-Agent": self.ua.random if self.ua else default_ua,
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            "Accept-Language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
            "Connection": "keep-alive",
            "Cache-Control": "no-cache"
        }

    def build_url(self, offset):
        """Construit l'URL pour une page donnée."""
        return f"{self.base_url}&o=a{offset}"

    def make_request(self, url, retries=3):
        """Effectue une requête GET avec gestion des erreurs et mise à jour du User-Agent."""
        for attempt in range(retries):
            try:
                headers = self.get_headers()
                response = self.session.get(url, headers=headers, timeout=30)
                response.raise_for_status()
                return response.text
            except Exception as e:
                logger.warning(f"Requête échouée (tentative {attempt + 1}) : {str(e)}")
                if attempt == retries - 1:
                    raise
                time.sleep(randint(3, 7))

    def parse_restaurant(self, element):
        """Extrait les informations d'un restaurant."""
        try:
            name_elem = element.find('a', class_='Lwqic Cj b')
            return {
                'Nom': name_elem.text.strip() if name_elem else "N/A",
                'URL': urljoin(self.base_url, name_elem['href']) if name_elem and name_elem.has_attr('href') else "N/A"
            }
        except Exception as e:
            logger.error(f"Erreur lors de l'analyse d'un restaurant : {str(e)}")
            return None

    def scrape_page(self, offset):
        """Récupère les informations des restaurants sur une page spécifique."""
        url = self.build_url(offset)
        logger.info(f"Scraping des résultats à l'offset {offset} : {url}")

        try:
            content = self.make_request(url)
            soup = BeautifulSoup(content, 'html.parser')

            # Extraction des restaurants
            restaurant_divs = soup.find_all('div', class_='RfBGI')
            restaurants = []

            for div in restaurant_divs:
                data = self.parse_restaurant(div)
                if data:
                    restaurants.append(data)
                time.sleep(randint(1, 2))

            return restaurants
        except Exception as e:
            logger.error(f"Erreur lors du scraping de l'offset {offset} : {str(e)}")
            return []

    def scrape_all_pages(self, max_pages=50):
        """Récupère les informations des restaurants sur plusieurs pages."""
        all_restaurants = []
        empty_pages = 0

        for page_number in range(max_pages):
            offset = page_number * 30
            restaurants = self.scrape_page(offset)
            if restaurants:
                all_restaurants.extend(restaurants)
                empty_pages = 0
                logger.info(f"Offset {offset} : {len(restaurants)} restaurants trouvés.")
            else:
                empty_pages += 1
                if empty_pages >= 2:
                    logger.info("Arrêt du scraping après 2 pages vides consécutives.")
                    break
            time.sleep(randint(self.min_delay, self.max_delay))  # Pause entre les pages
            
        return pd.DataFrame(all_restaurants)

def main():
    base_url = "https://www.tripadvisor.fr/RestaurantSearch?geo=187265&sortOrder=popularity"
    scraper = TripAdvisorScraper(base_url)

    try:
        df = scraper.scrape_all_pages(max_pages=1)  # Scrape jusqu'à 50 pages
        if not df.empty:
            df.to_csv('restaurants_lyon.csv', index=False, encoding='utf-8-sig')
            logger.info(f"Scraping terminé avec succès : {len(df)} restaurants trouvés.")
        else:
            logger.error("Aucun restaurant trouvé.")
    except Exception as e:
        logger.error(f"Échec du scraping : {str(e)}")

if __name__ == "__main__":
    main()

2024-12-22 12:19:59,843 - INFO - Scraping des résultats à l'offset 0 : https://www.tripadvisor.fr/RestaurantSearch?geo=187265&sortOrder=popularity&o=a0
2024-12-22 12:20:44,310 - INFO - Offset 0 : 31 restaurants trouvés.
2024-12-22 12:20:48,313 - INFO - Scraping des résultats à l'offset 30 : https://www.tripadvisor.fr/RestaurantSearch?geo=187265&sortOrder=popularity&o=a30
2024-12-22 12:21:33,306 - INFO - Offset 30 : 31 restaurants trouvés.
2024-12-22 12:21:38,317 - INFO - Scraping des résultats à l'offset 60 : https://www.tripadvisor.fr/RestaurantSearch?geo=187265&sortOrder=popularity&o=a60


In [22]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent, FakeUserAgentError
import pandas as pd
import time
from random import randint
import logging

# Configuration du logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class TripAdvisorScraper:
    def __init__(self, base_url, restaurant_name=None):
        """
        Initialise le scraper.
        :param base_url: URL de base pour le scraping
        :param restaurant_name: Nom du restaurant pour filtrer les résultats (facultatif)
        """
        self.base_url = base_url
        self.restaurant_name = restaurant_name
        self.setup_user_agent()
        self.session = requests.Session()
        self.min_delay = 2
        self.max_delay = 5

    def setup_user_agent(self):
        """Initialise le User-Agent avec gestion des erreurs."""
        try:
            self.ua = UserAgent()
        except FakeUserAgentError:
            logger.warning("Impossible d'utiliser FakeUserAgent. Utilisation d'un User-Agent par défaut.")
            self.ua = None

    def get_headers(self):
        """Crée des en-têtes HTTP pour simuler un navigateur."""
        default_ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        return {
            "User-Agent": self.ua.random if self.ua else default_ua,
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            "Accept-Language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
            "Connection": "keep-alive",
            "Cache-Control": "no-cache"
        }

    def build_url(self, offset):
        """Construit l'URL pour une page donnée avec un nom de restaurant."""
        if self.restaurant_name:
            # Ajout du nom du restaurant et des paramètres supplémentaires
            query = self.restaurant_name.replace(' ', '+')
            return f"{self.base_url}?q={query}&geo=187265&ssrc=e&searchNearby=false&offset={offset}"
        else:
            raise ValueError("Un nom de restaurant est requis pour cette recherche.")

    def make_request(self, url, retries=3):
        """Effectue une requête GET avec gestion des erreurs."""
        for attempt in range(retries):
            try:
                response = self.session.get(url, headers=self.get_headers(), timeout=30)
                response.raise_for_status()
                return response.text
            except Exception as e:
                logger.warning(f"Requête échouée (tentative {attempt + 1}) : {str(e)}")
                if attempt == retries - 1:
                    raise
                time.sleep(randint(3, 7))

    def scrape_page(self, offset):
        """Récupère les informations des restaurants sur une page spécifique."""
        url = self.build_url(offset)
        logger.info(f"Scraping des résultats à l'offset {offset} : {url}")

        try:
            content = self.make_request(url)
            soup = BeautifulSoup(content, 'html.parser')

            main_div = soup.find('main')
            
            # main_div = BeautifulSoup(main_div, 'html.parser')
            # Extraction des restaurants
            restaurant_divs = main_div.find_all('div', class_='kgrOn o')
           
            restaurants = []

            for div in restaurant_divs:
                a_tag = div.find('a', class_='BMQDV _F Gv wSSLS SwZTJ')
                if a_tag:
                    href = a_tag['href']
                    name = ''.join(a_tag.stripped_strings)
                    # Supprimer les numéros en début de nom, s'il y en a
                    name = name.split(". ", 1)[-1]
                    restaurants.append({'name': name, 'href': href})

            return restaurants
        except Exception as e:
            logger.error(f"Erreur lors du scraping de l'offset {offset} : {str(e)}")
            return []

    def scrape_all_pages(self, max_pages=2):
        """Récupère les informations des restaurants sur plusieurs pages."""
        all_restaurants = []
        empty_pages = 0

        for page_number in range(max_pages):
            offset = page_number * 30
            restaurants = self.scrape_page(offset)
            if restaurants:
                all_restaurants.extend(restaurants)
                empty_pages = 0
                logger.info(f"Offset {offset} : {len(restaurants)} restaurants trouvés.")
            else:
                empty_pages += 1
                if empty_pages >= 2:
                    logger.info("Arrêt du scraping après 2 pages vides consécutives.")
                    break
            time.sleep(randint(self.min_delay, self.max_delay))  # Pause entre les pages
            
        return pd.DataFrame(all_restaurants)

def main():
    base_url = "https://www.tripadvisor.fr/Search"
    restaurant_name = "Bouchon Lyonnais"  # Nom du restaurant pour filtrer
    scraper = TripAdvisorScraper(base_url, restaurant_name)

    try:
        df = scraper.scrape_all_pages(max_pages=10)  # Scrape jusqu'à 10 pages
        if not df.empty:
            df.to_csv('restaurants_lyon.csv', index=False, encoding='utf-8-sig')
            logger.info(f"Scraping terminé avec succès : {len(df)} restaurants trouvés.")
        else:
            logger.error("Aucun restaurant trouvé.")
    except Exception as e:
        logger.error(f"Échec du scraping : {str(e)}")

if __name__ == "__main__":
    main()


2024-12-22 12:04:21,021 - INFO - Scraping des résultats à l'offset 0 : https://www.tripadvisor.fr/Search?q=Bouchon+Lyonnais&geo=187265&ssrc=e&searchNearby=false&offset=0
2024-12-22 12:04:21,364 - ERROR - Erreur lors du scraping de l'offset 0 : 'NoneType' object is not callable
2024-12-22 12:04:24,368 - INFO - Scraping des résultats à l'offset 30 : https://www.tripadvisor.fr/Search?q=Bouchon+Lyonnais&geo=187265&ssrc=e&searchNearby=false&offset=30
2024-12-22 12:04:24,678 - ERROR - Erreur lors du scraping de l'offset 30 : 'NoneType' object is not callable
2024-12-22 12:04:24,678 - INFO - Arrêt du scraping après 2 pages vides consécutives.
2024-12-22 12:04:24,678 - ERROR - Aucun restaurant trouvé.


In [16]:
from bs4 import BeautifulSoup

# Le HTML à analyser
html_content = '''
<div class="RfBGI"><span><a class="Lwqic Cj b" href="/Restaurant_Review-g187265-d28100759-Reviews-Bikube_Lyon_Restaurant-Lyon_Rhone_Auvergne_Rhone_Alpes.html" target="_blank">Bikube Lyon - Restaurant</a></span></div>
<div class="RfBGI"><span><a class="Lwqic Cj b" href="/Restaurant_Review-g187265-d16216238-Reviews-Angelo_Italian_Restaurant-Lyon_Rhone_Auvergne_Rhone_Alpes.html" target="_blank">Angelo | Italian Restaurant</a></span></div>
<div class="RfBGI"><span><a class="Lwqic Cj b" href="/Restaurant_Review-g187265-d15114321-Reviews-L_affreux_Jojo-Lyon_Rhone_Auvergne_Rhone_Alpes.html" target="_blank">1<!-- -->. <!-- -->L'affreux Jojo</a></span></div>
<div class="RfBGI"><span><a class="Lwqic Cj b" href="/Restaurant_Review-g187265-d18626103-Reviews-Table_Partage-Lyon_Rhone_Auvergne_Rhone_Alpes.html" target="_blank">2<!-- -->. <!-- -->Table &amp; Partage</a></span></div>
<div class="RfBGI"><span><a class="Lwqic Cj b" href="/Restaurant_Review-g187265-d23110895-Reviews-Frazarin-Lyon_Rhone_Auvergne_Rhone_Alpes.html" target="_blank">3<!-- -->. <!-- -->Frazarin Bistrot Franco Italien</a></span></div>
<div class="RfBGI"><span><a class="Lwqic Cj b" href="/Restaurant_Review-g187265-d2027277-Reviews-La_Bouteillerie-Lyon_Rhone_Auvergne_Rhone_Alpes.html" target="_blank">4<!-- -->. <!-- -->La Bouteillerie</a></span></div>
<div class="RfBGI"><span><a class="Lwqic Cj b" href="/Restaurant_Review-g187265-d19896976-Reviews-Empanadas_Club-Lyon_Rhone_Auvergne_Rhone_Alpes.html" target="_blank">5<!-- -->. <!-- -->Empanadas Club</a></span></div>
'''

# Création de l'objet BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Extraction des liens et des noms
restaurants = []
for a_tag in soup.find_all('a', class_='Lwqic Cj b'):
    href = a_tag['href']
    # Supprime les éventuels commentaires ou éléments inutiles dans le texte
    name = ''.join(a_tag.stripped_strings)
    restaurants.append({'name': name, 'href': href})

# Affichage des résultats
for restaurant in restaurants:
    print(f"Nom : {restaurant['name']}, Lien : {restaurant['href']}")


Nom : Bikube Lyon - Restaurant, Lien : /Restaurant_Review-g187265-d28100759-Reviews-Bikube_Lyon_Restaurant-Lyon_Rhone_Auvergne_Rhone_Alpes.html
Nom : Angelo | Italian Restaurant, Lien : /Restaurant_Review-g187265-d16216238-Reviews-Angelo_Italian_Restaurant-Lyon_Rhone_Auvergne_Rhone_Alpes.html
Nom : 1.L'affreux Jojo, Lien : /Restaurant_Review-g187265-d15114321-Reviews-L_affreux_Jojo-Lyon_Rhone_Auvergne_Rhone_Alpes.html
Nom : 2.Table & Partage, Lien : /Restaurant_Review-g187265-d18626103-Reviews-Table_Partage-Lyon_Rhone_Auvergne_Rhone_Alpes.html
Nom : 3.Frazarin Bistrot Franco Italien, Lien : /Restaurant_Review-g187265-d23110895-Reviews-Frazarin-Lyon_Rhone_Auvergne_Rhone_Alpes.html
Nom : 4.La Bouteillerie, Lien : /Restaurant_Review-g187265-d2027277-Reviews-La_Bouteillerie-Lyon_Rhone_Auvergne_Rhone_Alpes.html
Nom : 5.Empanadas Club, Lien : /Restaurant_Review-g187265-d19896976-Reviews-Empanadas_Club-Lyon_Rhone_Auvergne_Rhone_Alpes.html
